In [1]:
import numpy as np
import pandas as pd

from util_io import init, finalize, dump_conf
from util_cost import cal_total, n_people, family_id_choice_to_pref_cost
from util_cost import choices as family_pref
from util_check import deep_check

In [2]:
# constants #
N_families = 3000
N_days = 100
N_min_people = 121
N_max_people = 300
# constants #

# params #
path_init_conf =     '../output/m08-improved.csv'
path_dump_improved = '../output/m10-improved.csv' # lowest cost

In [3]:
assigned_day, family_on_day, occupancy = init(path_conf=path_init_conf)
etotal_low = cal_total(assigned_day, occupancy)
print('Init config cost:', etotal_low)

Read initial configs...
Read config completed.
Init config cost: 73610.22770050484


In [4]:
len(assigned_day)

5000

## Ortools MIP

In [5]:
from ortools.linear_solver import pywraplp

In [6]:
solver = pywraplp.Solver('', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
# solver = pywraplp.Solver('', pywraplp.Solver.CPLEX_MIXED_INTEGER_PROGRAMMING)

In [7]:
#solver.SetSolverSpecificParametersAsString('threads')

In [8]:
solver.SetNumThreads(8)

True

In [9]:
families = range(N_families)
days = range(1, N_days + 1)

In [10]:
# Possible choice for the family
# last choice is any day that is not on the family's preferred days
N_choices = family_id_choice_to_pref_cost.shape[1]
N_choices

11

In [11]:
n_people = n_people[:N_families]
n_people.sum()

12602

In [12]:
family_id_choice_to_pref_cost = family_id_choice_to_pref_cost[:N_families]

In [13]:
family_id_choice_to_pref_cost.shape

(3000, 11)

In [14]:
family_id_choice_to_pref_cost

array([[   0,   50,   86, ...,  544, 1440, 2236],
       [   0,   50,   86, ...,  544, 1440, 2236],
       [   0,   50,   77, ...,  508, 1205, 1802],
       ...,
       [   0,   50,   95, ...,  580, 1675, 2670],
       [   0,   50,  104, ...,  616, 1910, 3104],
       [   0,   50,   95, ...,  580, 1675, 2670]])

In [15]:
family_pref = family_pref[:N_families]

## Variables

In [16]:
# Variables
# assignment matrix[family, pref_rank]
assignment_matrix = {}
for family in families:
    for c in range(N_choices):
        assignment_matrix[family, c] = solver.BoolVar('x[%i,%i]' % (family, c))

In [17]:
len(assignment_matrix)

33000

In [18]:
possible_family_sizes = np.unique(n_people)

In [19]:
# unpreferred_day_counts[day, size]
unpreferred_day_counts = {}
for day in days:
    for size in possible_family_sizes:
        ub = int(N_max_people / size)
        # ub = solver.infinity()
        unpreferred_day_counts[day, size] = solver.IntVar(0, ub, 'd[%i,%i]' % (day, size))

In [20]:
len(unpreferred_day_counts)

700

## Constraints

### constraint 1: each family only take one day (choice)

In [21]:
# Constraints
# constraint 1: each family only take one day (choice)
for family in families:
    solver.Add(solver.Sum([assignment_matrix[family, c]
                           for c in range(N_choices)]) == 1)

### constraint 2: each day can only have 125-300 people

In [22]:
# constraint 2: each day can only have 125-300 people

In [23]:
N_family_pref = N_choices - 1
N_family_pref

10

In [24]:
# family_pref = np.array([np.random.choice(N_days, size=(N_family_pref, ), replace=False) + 1 
#                         for _ in range(N_families)]) # reduce problem for testing

In [25]:
family_pref

array([[ 52,  38,  12, ...,  76,  10,  28],
       [ 26,   4,  82, ...,   6,  66,  61],
       [100,  54,  25, ...,  89,  80,  33],
       ...,
       [  1,  34,  75, ...,  29,  16,  33],
       [ 42,  88,   1, ...,  69,   9,   5],
       [ 98,  40,  75, ...,  88,  24,  96]])

In [26]:
family_pref.shape

(3000, 10)

In [27]:
# day to dictionary of families who choice this day with value as preference rank
days_family_prefered = [{} for day in range(N_days+1)]  # day = 0 should not be used

In [28]:
for family, pref in enumerate(family_pref):
    for pref_rank, day in enumerate(pref):
        days_family_prefered[day][family] = pref_rank

In [29]:
for day in days:
    # find those family who like this day
    family_prefered = days_family_prefered[day]
    solver.Add(
        solver.Sum(
            [assignment_matrix[family, pref_rank] * n_people[family] 
             for family, pref_rank in family_prefered.items()]
        )
        + solver.Sum(
            [unpreferred_day_counts[day, s] * s for s in possible_family_sizes]
        )
        <= N_max_people,
        'day_upper_bound[%i]' % day
    )
    solver.Add(
        solver.Sum(
            [assignment_matrix[family, pref_rank] * n_people[family] 
             for family, pref_rank in family_prefered.items()]
        )
        + solver.Sum(
            [unpreferred_day_counts[day, s] * s for s in possible_family_sizes]
        )        
        >= N_min_people,
        'day_lower_bound[%i]' % day
    )

### constraint 3: unpreferred day family count conservation for each family size

In [30]:
# constraint 3: unpreferred day family count conservation for each family size

In [31]:
family_size_to_family_ids = {
    size: np.where(n_people == size)[0] for size in possible_family_sizes
}

In [32]:
for size in possible_family_sizes:
    solver.Add(
        solver.Sum([assignment_matrix[family, N_choices - 1]
                    for family in family_size_to_family_ids[size]])
        == solver.Sum([unpreferred_day_counts[day, size] for day in days]),
        'unpreferred_day_counts[%i]' % size
    )

## Objective

In [33]:
# Objective - Preference cost only as approximation
solver.Minimize(
    solver.Sum([
        assignment_matrix[family, c] * family_id_choice_to_pref_cost[family, c]
        for family in families for c in range(N_choices)
    ])
)

xx = solver.constraints()[0]

[xx.GetCoefficient(assignment_matrix[1, c]) for c in range(N_choices)]

xx.name()

## Solve

In [34]:
len(solver.variables())

33700

In [39]:
%%time
# Solve
sol = solver.Solve()

resdict = {0:'OPTIMAL', 1:'FEASIBLE', 2:'INFEASIBLE', 3:'UNBOUNDED', 
           4:'ABNORMAL', 5:'MODEL_INVALID', 6:'NOT_SOLVED'}
print('status: ', resdict[sol])
print('Total cost = ', solver.Objective().Value())
print("Time = ", solver.WallTime(), " milliseconds")

status:  OPTIMAL
Total cost =  73177.0
Time =  52704  milliseconds
CPU times: user 52.3 s, sys: 3.62 s, total: 55.9 s
Wall time: 19.9 s


In [36]:
# 3000, min=121
# 12: 20.1, 22.1
# 8: 21 19.9
# 6: 18.6 19.5
# 4: 23 21.5

# 2000, min=80
# 12: 13060  milliseconds
# 8: 12339  milliseconds
# 7: 12338  milliseconds
# 6: 11656  milliseconds
# 5: 12287  milliseconds
# 4: 12035  milliseconds

## Debug

In [37]:
# [
#     [assignment_matrix[family, c].solution_value() for c in range(N_choices)]
#     for family in range(10)
# ]        

In [38]:
# [
#     [unpreferred_day_counts[day, size].solution_value() for size in possible_family_sizes]
#     for day in range(1, 10)
# ]        